# Active Inference: Multi-Factor Generative Model
This notebook simulates an active inference agent behaving in a random environment described by two hidden state variables and a single observation modality. The agent uses variational inference to infer the most likely hidden states, and optimizes its policies with respect to those that minimize the expected free energy of their attendant observations.

## Import basic paths

In [29]:
import os
import sys
from pathlib import Path
path = Path(os.getcwd())
module_path = str(path.parent) + '/'
sys.path.append(module_path)

## Import `inferactively` module

In [30]:
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import special

from inferactively.distributions import Categorical, Dirichlet
from inferactively import functions as F

## Define an auxiliary function for creating the transition likelihood

In [31]:
def create_B(Ns, Nf, controllableActionIdx):
    """
    Generate controlled transitions for each hidden state factor, that correspond to actions.
    """

    B = np.empty((Nf),dtype=object)
    for si, ndim_si in enumerate(Ns):
        B[si] = np.eye(ndim_si)

    # controllable hidden state factors - transition to the k-th location

    for pi in controllableActionIdx:
        B[pi] = np.tile(B[pi].reshape(Ns[pi],Ns[pi],1),(1,1,Ns[pi])).transpose((1,2,0))
    
    return B

## The generative process
Here, we setup the mechanics of the environment, or the 'generative process.' To make this analogous to the generative _model_ learned by the agent, we describe these mechanics using likelihood distribution $P(o_t|s_t)$, denoted `A_GP`, and a transition distribution $P(s_t|s_{t-1},a_{t-1})$, denoted `B_GP`. The generative process will be used to generate observations `obs` via the likelihood $P(o_t|s_t)$ and is changed by actions via the likelihood $P(s_t|s_{t-1},a_{t-1})$.

In [32]:
# set up state-space and outcome-space dimensionalities of the generative process
No = [4]     # dimensionality of the different outcome modalities
Ng = len(No) # total number of outcome modalities

Ns = [3, 2]  # dimensionality of the hidden state factors
Nf = len(Ns) # total number of hidden state factors

# Create the likelihoods and priors relevant to the generative model
if Ng == 1:
    A_GP = Categorical(values = np.random.rand(*(No+Ns)))
    A_GP.normalize()
else:
    A_GP = np.empty(Ng, dtype = object)
    for g in range(Ng):
        A_GP[g] = np.random.rand(*(No[g] + Ns))
    A_GP = Categorical(values = A_GP)
    A_GP.normalize()

B_GP = Categorical(values = create_B(Ns, Nf, [0, 1]) )

initState = np.array( [np.eye(Ns[f])[np.random.randint(Ns[f])] for f in range(Nf) ], dtype = object)

T = 100

## The generative model
Here, we setup the belief structure of the active inference agent, or the 'generative model.' For this simple case, we make the generative model identical to the generative process. Namely, the agent's beliefs about the observation and likelihood distributions (respectively, the _observation model_ and _transition model_ ) are identical to the true parameters describing the environment.

In [33]:
# Generative model likelihoods
A_GM = Categorical(values = A_GP.values) # in this case, the generative model and the generative process are identical
B_GM = Categorical(values = B_GP.values) # in this case, the generative model and the generative process are identical

# Prior Dirichlet parameters (these parameterize the generative model likelihoods)
pA = Dirichlet(values = A_GM.values * 1e20) # fix prior beliefs about observation likelihood to be really high (and thus impervious to learning)
pB = Dirichlet(values = B_GP.values * 1e20) # fix prior beliefs about transition likelihood to be really high (and thus impervious to learning)

# create some preference over observations

if Ng == 1:
    C = np.zeros(*No)
    C[0] = -2 # prefers not to observe the outcome with index == 0
    C[-1] = 2 # prefers to observe the outcome with highest index
else:
    C = np.empty(Ng, dtype = object)
    for g in range(Ng):
        C[g] = np.zeros(No[g])
    C[0][0] = -2
    C[0][-1] = 2

prior = Categorical(values = np.array([np.ones(Ns[f])/Ns[f] for f in range(Nf)], dtype = object))

# policy related parameters
policy_horizon = 1
cntrl_fac_idx = [0, 1]
Nu, possiblePolicies = F.constructNu(Ns,Nf,cntrl_fac_idx,policy_horizon)

# Action-Perception Loop

## Initialize history of beliefs, hidden states, and observations

In [34]:
# %% Run the action perception loop
s = initState

# set up some variables to store history of actions, etc.
actions_hist = np.zeros( (len(Nu),T) )
states_hist = np.zeros( (Nf,T) )
obs_hist = np.zeros( (Ng,T) )
Qs_hist = np.empty(Nf, dtype=object)
for f in range(Nf):
    Qs_hist[f] = np.zeros( (Ns[f], T) )

## Main loop over time

In [35]:
for t in range(T):

    #### STEP ONE: SAMPLE AN OBSERVATION FROM THE GENERATIVE PROCESS ######
    ps = A_GP.dot(s)
    obs = ps.sample()

    #### STEP TWO: INVERT GENERATIVE MODEL TO INFER MOST LIKELY HIDDEN STATE ######

    Qs = F.update_posterior_states(A_GM, obs, prior, return_numpy = False)

    #### STEP THREE: INFER THE MOST LIKELY POLICIES (USING EXPECTED FREE ENERGY ASSUMPTION) #####

    Q_pi,EFE = F.update_posterior_policies(Qs, A_GM, pA, B_GM, pB, C, possiblePolicies, gamma = 16.0, return_numpy=True)

    #### STEP FOUR: SAMPLE AN ACTION FROM THE POSTERIOR OVER CONTROLS, AND PERTURB THE GENERATIVE PROCESS USING THE SAMPLED ACTION #####
    action = F.sample_action(Q_pi, possiblePolicies, Nu, sampling_type = 'marginal_action')

    s = np.array( [B_GP[f][:,:,a_i].dot(s[f],return_numpy=True) for f, a_i in enumerate(action)], dtype=object) 
    
     #### STORE VARIABLES IN HISTORY ####
    actions_hist[:,t] = np.array(action)
    for f in range(Nf):
        states_hist[f,t] = np.where(s[f])[0]

    obs_hist[:,t] = np.array(obs)
    
    for f in range(Nf):
        Qs_hist[f][:,t] = Qs[f].values[:,0].copy()